#Kagle Titanic

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt

In [ ]:
%pylab inline

In [ ]:
np.random.seed(525)

##Loading the Data

In [ ]:
train = pd.read_csv('data/train.csv')
train.info()

In [ ]:
train[:10]

In [ ]:
train.describe()

In [ ]:
train.Survived.value_counts()

In [ ]:
test = pd.read_csv('data/test.csv')
test.info()

In [ ]:
test[:10]

In [ ]:
test.describe()

In [ ]:
test['Survived'] = np.nan

In [ ]:
d = pd.concat([train, test])
d.info()

In [ ]:
d.describe()

In [ ]:
len(d)

In [ ]:
d.index = pd.Index(range(len(d)))
d[-10:]

##Feature Engineering

In [ ]:
d[:10]

###Title

In [ ]:
import re
p = re.compile("[\.,]")
p.split(d.Name[0])[1].strip()

In [ ]:
d['Title'] = d.Name.str.split("[\.,]").apply(lambda l: l[1]).str.strip().replace(" ", "")
d.Title.value_counts()

In [ ]:
d.loc[d.Title.isin(['Mme', 'Mlle']),'Title'] = 'Mlle'
d.loc[d.Title.isin(['Capt', 'Don', 'Major', 'Sir']),'Title'] = 'Sir'
d.loc[d.Title.isin(['Dona', 'Lady', 'the Countess', 'Jonkheer']),'Title'] = 'Lady'
d.Title.value_counts()

###Family Size

In [ ]:
d.FamilySize = d.SibSp + d.Parch + 1
d.FamilySize.value_counts()

###Family ID

In [ ]:
d.Surname = d.Name.str.split("[\.,]").apply(lambda l: l[0]).str.strip().replace(" ", "")
d.Surname[:10]

In [ ]:
d['FamilyID'] = d.Surname.str.cat(d.FamilySize.apply(lambda n: str(n)))
d.FamilyID[:10]

In [ ]:
d.loc[d.FamilySize <= 2, 'FamilyID'] = 'Small'
d.FamilyID.value_counts()[-10:]

In [ ]:
smallFamIDs = d.FamilyID.value_counts()[famIDs <= 2].index.tolist()
smallFamIDs[:10]

In [ ]:
d.loc[d.FamilyID.isin(smallFamIDs), 'FamilyID'] = 'Small'
d.FamilyID.value_counts()[-10:]

In [ ]:
d[-10:]

###Encode Categorical Features

In [ ]:
title_dummies = pd.get_dummies(d.Title, prefix="Title")
d = pd.concat([d, title_dummies], axis=1)
del d['Title']

In [ ]:
famid_dummies = pd.get_dummies(d.FamilyID, prefix="FamilyID")
d = pd.concat([d, famid_dummies], axis=1)
del d['FamilyID']

In [ ]:
d.loc[pd.isnull(d.Embarked), 'Embarked'] = 'S'
embarked_dummies = pd.get_dummies(d.Embarked, prefix="Embarked")
d = pd.concat([d, embarked_dummies], axis=1)
del d['Embarked']

In [ ]:
sex_dummies = pd.get_dummies(d.Sex, prefix="Sex")
d = pd.concat([d, sex_dummies], axis=1)
del d['Sex']

In [ ]:
del d['Name']
del d['Cabin']
del d['Ticket']

###Fill in Missing Values

In [ ]:
d.info()

In [ ]:
d.loc[pd.isnull(d.Fare), 'Fare'] = d.Fare.mean()

In [ ]:
d_labels = d.loc[:, 'Survived']
del d['Survived']

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor()
age_train = d.loc[pd.notnull(d.Age)]
age_train_target = age_train.Age
del age_train['Age']
clf.fit(age_train, age_train_target)

In [ ]:
len(d.loc[pd.notnull(d.Age)])

In [ ]:
len(d.loc[pd.isnull(d.Age)])

In [ ]:
age_unknown = d[pd.isnull(d.Age)]
del age_unknown['Age']
d.loc[pd.isnull(d.Age), 'Age'] = clf.predict(age_unknown)
d.info()

##Split the Data

In [ ]:
d['Survived'] = d_labels
train = d.loc[pd.notnull(d['Survived']),:]
train_labels = train.loc[:, 'Survived']
del train['Survived']
test = d.loc[pd.isnull(d['Survived']),:]
del test['Survived']

##Predict

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=2000, max_depth=10, oob_score=True)
clf.fit(train, train_labels)
clf.score(train, train_labels)

In [ ]:
res = clf.predict(test)

In [ ]:
submission = DataFrame({'PassengerId': np.arange(892, 1310),'Survived': res})
submission.Survived = submission.Survived.astype(int)
submission.to_csv('rf_02.csv', index=False)